In [1]:
import pandas as pd

dataset: https://www.kaggle.com/datasets/mirichoi0218/insurance

In [4]:
import kagglehub

/Users/karthiksridhar/anaconda3/envs/mlops_new_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import kaggle

kaggle.api.authenticate()

kaggle.api.dataset_download_files('mirichoi0218/insurance', path='dataset', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/mirichoi0218/insurance


In [15]:
df = pd.read_csv('dataset/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [16]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [17]:
df.shape

(1338, 7)

In [24]:
cols = list(df.columns)
print(cols)

['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']


In [35]:
[(f'{col}: {df[col].count()}') for col in cols]


['age: 1338',
 'sex: 1338',
 'bmi: 1338',
 'children: 1338',
 'smoker: 1338',
 'region: 1338',
 'charges: 1338']

In [36]:
[(f'{col}: {df[col].nunique()}') for col in cols]


['age: 47',
 'sex: 2',
 'bmi: 548',
 'children: 6',
 'smoker: 2',
 'region: 4',
 'charges: 1337']

In [38]:
[(f'{col}: {df[col].dtype}') for col in cols]

['age: int64',
 'sex: object',
 'bmi: float64',
 'children: int64',
 'smoker: object',
 'region: object',
 'charges: float64']

In [39]:
[(f'{col}: {pd.api.types.is_numeric_dtype(df[col])}') for col in cols]


['age: True',
 'sex: False',
 'bmi: True',
 'children: True',
 'smoker: False',
 'region: False',
 'charges: True']

In [40]:
[(f'{col}: {df[col].isnull().sum()}') for col in cols]


['age: 0',
 'sex: 0',
 'bmi: 0',
 'children: 0',
 'smoker: 0',
 'region: 0',
 'charges: 0']

In [60]:
import numpy as np
from scipy import stats as st

In [65]:
#functions to get basic stats per feature
functions =  {
  "count": lambda x: x.count(),
  "min": lambda x: x.min(),
  "max": lambda x: x.max(),
  "1st quartile": lambda x: x.quantile(.25),
  "2nd quartile": lambda x: x.quantile(.50),
  "3rd quartile": lambda x: x.quantile(.75),
  "mean": lambda x: x.mean(),
  "median": lambda x: x.median(),
  "mode": lambda x: x.mode()
}

fns = list(functions.keys())
feature = "charges"
[print(f"{feature} {name}: {functions[name](df[feature])}") for name in fns[:-1]]

print(f"{feature} {fns[-1]}: {st.mode(df[feature])[0]}")

charges count: 1338
charges min: 1121.8739
charges max: 63770.42801
charges 1st quartile: 4740.28715
charges 2nd quartile: 9382.033
charges 3rd quartile: 16639.912515
charges mean: 13270.422265141257
charges median: 9382.033
charges mode: 1639.5631


In [67]:
print(df.charges.std()) #assumes sample

12110.011236694001


In [69]:
print(np.std(df.charges)) #assumes population

12105.484975561612


In [71]:
print(np.std(df.charges, ddof=1))

12110.011236694001


In [75]:
from scipy.stats import kurtosis, skew
#false accounts for stastical bias in calculating sample mean instead of population mean
print(skew(df.charges, bias=False))
print(kurtosis(df.charges, bias = False))

1.5158796580240383
1.6062986532967916


In [88]:
df_numerical = df.drop(["sex", "smoker", "region"], axis =1)
df_numerical.corr()

,age,bmi,children,charges
age,1.000000,0.109272,0.042469,0.299008
bmi,0.109272,1.000000,0.012759,0.198341
children,0.042469,0.012759,1.000000,0.067998
charges,0.299008,0.198341,0.067998,1.000000


In [90]:
df.charges.corr(df.bmi)

np.float64(0.19834096883362884)

In [101]:
from scipy import stats as st
r, p = st.pearsonr(df.charges, df.age)
print(round(r, 4), round(p, 29))

0.299 5.0000000000000004e-29


In [ ]:
corr_df = pd.DataFrame(columns=['r','p'])

for col in df_numerical:
  r,p = st.pearsonr(df.charges, df[col])
  corr_df.loc[col] = [round(r,3), round(p,3)]